In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
df = pd.read_excel('date_it_lemm.xlsx')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         2308 non-null   int64 
 1   title        2308 non-null   object
 2   link         2308 non-null   object
 3   annotation   2308 non-null   object
 4   authors      2308 non-null   object
 5   text         2308 non-null   object
 6   UDK          1558 non-null   object
 7   ESCI         2308 non-null   int64 
 8   GeoRef       2308 non-null   int64 
 9   ВАК          2308 non-null   int64 
 10  -            2308 non-null   int64 
 11  RSCI         2308 non-null   int64 
 12  WOS          2308 non-null   int64 
 13  AGRIS        2308 non-null   int64 
 14  PubMed       2308 non-null   int64 
 15  MathSciNet   2308 non-null   int64 
 16  Scopus       2308 non-null   int64 
 17  zbMATH       2308 non-null   int64 
 18  CAS          2308 non-null   int64 
 19  count_words  2308 non-null 

In [29]:
df = df.assign(authors=df['authors'].str.split(', ')).explode('authors') #разделяем авторов по запятой, записываем каждому авторы его статью
#даже если это дубликат
df[0:3]

,date,title,link,annotation,authors,text,UDK,ESCI,GeoRef,ВАК,...,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS,count_words,topic
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,проблема обработка данные георадиолокационный ...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,776,IT
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,Zemtsov Andrey Nikolaevich,проектирование современный высоконадёжный сеть...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1421,IT
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,Tran Dung Khanh,проектирование современный высоконадёжный сеть...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1421,IT


In [21]:
grouped_df = df.groupby('authors').agg({
    'title': lambda x: ' | '.join(x),
    'annotation': lambda x: ' | '.join(x),
    'text': lambda x: ' | '.join(x)
}).reset_index()

grouped_df[1000:1010] #здесь нет проверки на тёзок, но с теми данными которые вытаскиваются с киберленинки её не добавить
#если парсить сайт, где есть принадлежность к организации (например, елайбрари), можно сверять годы работы и организацию, и если есть совпадение
#то объединять. ещё если из разных областей науки соавторов искать, можно добавить умную проверку 
#на схожесть тем у одного человека, но тут и так тема одна у всех

,authors,title,annotation,text
1000,Гольдштейн Александр Борисович,ТРАНСФОРМАЦИЯ ПОДХОДОВ К УПРАВЛЕНИЮ СОВРЕМЕННЫ...,Современные сети связи специального назначения...,трансформация подход управление современный се...
1001,Гонтарь Дмитрий Николаевич,Концептуальные подходы к формированию сценарие...,Актуальность: одной из наиболее значимых персп...,система управление связь безопасность концепту...
1002,Гончар Артем Александрович,Анализ проблем моделирования автоматизированно...,Постановка задачи: одной из основных тенденций...,система управление связь безопасность анализ п...
1003,Гончар Дмитрий Русланович,Оптимизация структуры базы данных реального вр...,Исследуется задача оптимизация структуры базы ...,ббк оптимизация структура база данные реальный...
1004,Гончаренко Владимир Иванович,Планирование съемки множества спортивных мероп...,Решаемая в настоящей работе задача относится к...,планирование съёмка множество спортивный мероп...
1005,Гончаров А. Л.,Основные этапы идентификации системы ускоренно...,no annotation,основной этап идентификация система ускоренный...
1006,Гончаров Д. В.,ОПИСАНИЕ ПРОЦЕССА ПРОЕКТИРОВАНИЯ АВТОМАТИЗИРОВ...,В данной статье описывается процесс проектиров...,свиридов ассистент кафедра прикладной информат...
1007,Гончарова О. В.,Концептуальные подходы к разработке автоматизи...,В статье рассматривается и анализируется опыт ...,берсенев михайлов далее ю черкасов н кудрин г ...
1008,Гопиржанов М.,ИСПОЛЬЗОВАНИЕ СИНЕРГЕТИЧЕСКОГО ПОДХОДА К ИССЛЕ...,В данной статье приведена методика определения...,хашимов доктор экономический наука доцент кафе...
1009,Горбань Г. В.,ІНТЕГРАЦіЯ ТЕХНОЛОГіЙ OLAP ТА DATA MINING ПРИ ...,Представлено особливості пошуку асоціативних п...,ка нотатка м жвуз струтинський б статистичный ...


In [32]:
df = grouped_df

#### Функция для вычисления близких авторов:

In [143]:
def recommend_similar_authors(author_name, df, cosine_sim, top_n=3):
    # Находим индекс автора
    author_indices = df[df['authors'] == author_name].index
    
    # Собираем все статьи автора
    author_articles = df.loc[author_indices]
    
    # Находим индексы статей, которые наиболее близки к статьям автора
    similar_articles = []
    for i in author_indices:
        sim_scores = list(enumerate(cosine_sim[i]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # Исключаем саму статью и соавторов (близость = 1)
        sim_scores = [j for j in sim_scores if j[1] < 0.99]
        for j, score in sim_scores:
            similar_articles.append((j, score))
    
    # Группируем авторов по статьям и сохраняем коэффициент схожести
    similar_authors = {}
    for article_index, score in similar_articles:
        article_title = df.iloc[article_index]['title']
        authors = df[df['title'] == article_title]['authors'].tolist()
        # Исключаем целевого автора и его соавторов
        authors = [a for a in authors if a != author_name]
        if authors:
            if article_title not in similar_authors:
                similar_authors[article_title] = {
                    'authors': authors,
                    'similarity_score': score
                }
    
    # Формируем список рекомендаций
    recommendations = []
    for article_title, data in similar_authors.items():
        recommendations.append({
            'article_title': article_title,
            'authors': data['authors'],
            'similarity_score': data['similarity_score']
        })
    
    # Сортируем рекомендации по коэффициенту схожести
    recommendations = sorted(recommendations, key=lambda x: x['similarity_score'], reverse=True)
    
    # Ограничиваем количество рекомендаций до top_n
    return recommendations[:top_n]

#### Функция для теста:

In [145]:
def test_recommendation_system(author_name, df, cosine_sim, top_n=3):
    # Рекомендуем похожих авторов
    recommendations = recommend_similar_authors(author_name, df, cosine_sim, top_n)
    
    # Выводим статьи и аннотации для автора на входе
    print(f"Статьи и аннотации автора {author_name}:")
    author_data = df[df['authors'] == author_name]
    for _, row in author_data.iterrows():
        print(f"- Заголовок: {row['title']}")
        print(f"  Аннотация: {row['annotation']}")
    print("\n")
    
    # Выводим рекомендации
    print(f"Рекомендуемые авторы для {author_name}:")
    for i, rec in enumerate(recommendations, 1):
        print(f"Рекомендация {i}:")
        print(f"- Статья: {rec['article_title']}")
        print(f"  Рекомендуемые соавторы: {', '.join(rec['authors'])}")
        print(f"  Коэффициент схожести: {rec['similarity_score']:.2f}")
        print("\n")

### 1. TF-IDF

In [38]:
vectorizer = TfidfVectorizer() #векторизуем
tfidf_matrix = vectorizer.fit_transform(df['text'])

In [44]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) #косинусное сходство между векторами

In [130]:
df[1078:1079]

,authors,title,annotation,text
1078,Гришин Сергей,Алгоритм преобразования частоты кадров на осно...,В данной статье предлагается алгоритм двукратн...,алгоритм преобразование частота кадр основа ин...


In [152]:
author_name = 'Гришин Сергей'  # Введите ФИО автора
test_recommendation_system(author_name, df, cosine_sim, top_n=3)

Статьи и аннотации автора Гришин Сергей:
- Заголовок: Алгоритм преобразования частоты кадров на основе интерполяции скомпенсированных кадров
  Аннотация: В данной статье предлагается алгоритм двукратного увеличения частоты кадров видеопоследовательности, использующий двунаправленную компенсацию движения для создания новых интерполированных кадров. Отличительной особенностью алгоритма является выполнение преобразования без использования сглаживания...


Рекомендуемые авторы для Гришин Сергей:
Рекомендация 1:
- Статья: Анализ применимости алгоритма фазовой корреляции при стабилизации последовательностей видеокадров капиллярного кровотока
  Рекомендуемые соавторы: Волков М. В., Каримов К. А.
  Коэффициент схожести: 0.39


Рекомендация 2:
- Статья: Применение вейвлет-преобразования для внедрения ЦВЗ в видеопоток в режиме реального времени
  Рекомендуемые соавторы: Григорьян Амаяк Карэнович, Литвинов М. Ю.
  Коэффициент схожести: 0.35


Рекомендация 3:
- Статья: РЕАЛИЗАЦИЯ АЛГОРИТМА СТАБИЛИ

Всё хорошо.

In [147]:
author_name = 'Баклан И. В.'  # Введите ФИО автора
test_recommendation_system(author_name, df, cosine_sim, top_n=3)

Статьи и аннотации автора Баклан И. В.:
- Заголовок: Использование лингвистического моделирования для аутентификации по динамике движений пользователя | Математические модели прогнозирования временных рядов различной природы
  Аннотация: В статье рассматривается задание о решении задачи создания элементов защиты информационной системы для идентификации пользователя по динамике движений манипулятором "мышь" или пальцем, или стилусом по сенсорному экрану. | В статье представлен обзор современных математических методов для прогнозирования временных рядов различной природы, в том числе для прикладного применения выбраны временные ряды экономического, экологического и социального типов. Предложенный универсальный аппарат прогнозирования базируется на...


Рекомендуемые авторы для Баклан И. В.:
Рекомендация 1:
- Статья: Математические модели прогнозирования временных рядов различной природы
  Рекомендуемые соавторы: Шулькевич Т. В.
  Коэффициент схожести: 0.83


Рекомендация 2:
- Статья: Исп

Тут проблема, если у одного человека 2 и больше статей, то первыми в рекомендации высвечиваются его же соавторы на его 2 или больше статей (но у соавторов то в ячейке одна статья, поэтому коэффициент не 1 и не отметается)

Идея для улучшения: у датафрейма ещё сделать столбик с количеством статей у 1 человека (можно по разделителям посчитать)